# NLG: Realiser

En lingúística (en concreto, en NLG, o Natural Language Generation) , la _realización_ es el proceso mediante el cual se genera texto real en un determinado idioma a partir de una representación sintáctica.

Sin entrar en demasiados detalles, las principales etapas en un modelo de NLG son:

1. Planificación del documento: decidir qué se quiere "decir" y construir un documento absracto que describa la estructura de la información que se presentará.
2. Microplanning: elección de palabras y expresiones para desarrollar las especificaciones del documento final.
3. Realización: convertir todas las especificaciones del texto abstracto en texto real. Habrá que tener en cuenta:
    - Reglas sintácticas
    - Reglas morfológicas 
    - Reglas ortográficas

<img src=https://www.researchgate.net/profile/Kamenka_Staykova/publication/272264342/figure/fig2/AS:359792108425219@1462792531151/Sequential-architecture-of-NLG-system.png width=650px>

Realizaremos algunos ejemplos usando [SimpleNLG-ES](https://github.com/citiususc/SimpleNLG-ES), API sobre Java para NLG.

[nlglib](https://github.com/roman-kutlak/nlglib/wiki/Intro) es una librería open source para Python que ofrece capacidades para la Generación Automática de Texto. Actualmente realiza dicha generación mediante lógica de primer orden.

## Logging

In [2]:
# !pip install --upgrade nlglib

In [29]:
import logging
import sys

level = logging.INFO

root = logging.getLogger()
root.setLevel(level)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(level)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

## Levantamos el server

In [30]:
from nlglib.realisation.simplenlg.client import SimpleNLGServer

In [34]:
port = 4020
server = SimpleNLGServer('../../data/realiser_lib/SimpleNLG-es-1.1.0.jar', port)
server.start()

2020-07-07 19:55:55,065 - nlglib.simplenlg.server - ERROR - java.net.BindException: Address already in use (Bind failed)
2020-07-07 19:55:55,065 - nlglib.simplenlg.server - ERROR - java.net.BindException: Address already in use (Bind failed)
2020-07-07 19:55:55,076 - nlglib.simplenlg.server - ERROR - 	at java.net.PlainSocketImpl.socketBind(Native Method)
2020-07-07 19:55:55,076 - nlglib.simplenlg.server - ERROR - 	at java.net.PlainSocketImpl.socketBind(Native Method)
2020-07-07 19:55:55,080 - nlglib.simplenlg.server - ERROR - 	at java.net.AbstractPlainSocketImpl.bind(AbstractPlainSocketImpl.java:387)
2020-07-07 19:55:55,080 - nlglib.simplenlg.server - ERROR - 	at java.net.AbstractPlainSocketImpl.bind(AbstractPlainSocketImpl.java:387)
2020-07-07 19:55:55,082 - nlglib.simplenlg.server - ERROR - 	at java.net.ServerSocket.bind(ServerSocket.java:375)
2020-07-07 19:55:55,082 - nlglib.simplenlg.server - ERROR - 	at java.net.ServerSocket.bind(ServerSocket.java:375)
2020-07-07 19:55:55,091 - nl

In [35]:
# Para tumbar el server
# server.shutdown()

# Realiser

In [36]:
from nlglib.realisation.simplenlg.realisation import Realiser
realiser = Realiser(host='localhost', port=port)

## Estructuras

In [37]:
from nlglib.microplanning import Clause, NP, VP, Adjective, PrepositionPhrase, Coordination, Conjunction

### Construir una frase

In [58]:
p = Clause("María", "perseguir", "un mono")

print(realiser(p))

María persigue un mono.


In [53]:
p['TENSE'] = 'FUTURE'

realiser(p)

'María perseguirá un mono.'

In [54]:
p['TENSE'] = 'PAST'

realiser(p)

'María persiguió un mono.'

In [61]:
p['PASSIVE'] = 'TRUE'
realiser(p)

'Un mono se perseguido por María.'

In [62]:
p['INTERROGATIVE_TYPE'] = 'YES_NO'
realiser(p)

'¿Se un mono perseguido por María?'

In [63]:
p['NEGATED'] = 'true'
realiser(p)

'¿No se un mono perseguido por María?'

### Construirlo elemento a elemento

In [66]:
c = Clause()

subject = NP("la", "corredora")
objekt = NP("un", "mono")
verb = VP("perseguir")

subject.premodifiers.append("rápida")
subject += Adjective('lista')

c.subject = subject
c.predicate = verb
c.object = objekt
# c['TENSE'] = 'PRESENT'

#c['NEGATED'] = 'true'
#c['INTERROGATIVE_TYPE'] = 'YES_NO'

print(realiser(c))

La rápida, lista corredora persigue un mono.


## Unir oraciones

In [67]:
p = Clause("yo", "estar", "feliz");
q = Clause(None, "comer", "pescado");
# q = Clause(None, verb="comer", object="pescado");

In [68]:
q['COMPLEMENTISER'] = "porque"
q['TENSE'] = 'PAST'

p.complements.append(q)

In [69]:
realiser(p)

'Yo estoy feliz porque comió pescado.'